# Импорт Библиотек

In [179]:
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

# Установим устройство (CPU или GPU)

In [180]:
torch.cuda.is_available(), torch.backends.cudnn.enabled

(True, True)

In [181]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Импорт Данных

То, на чем будем учиться

In [182]:
train = pd.read_csv('train.csv')

In [183]:
# Всего 42_000
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


То, на чем будем сдавать в самом конце для kaggle

In [184]:
finaltest = pd.read_csv('test.csv')

In [185]:
finaltest.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Пример того, как надо сдавать результаты

In [186]:
sample_submission = pd.read_csv('sample_submission.csv')

In [187]:
sample_submission.head()

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


# Подготовка Данных

Извлекаем отдельно признаки и отдельно целевые значения

In [188]:
features_np = train.iloc[:, 1:].values
targets_np = train.label.values

Производим нормировку признаков

In [189]:
features_norm_np = features_np / 255

In [190]:
# Проверка, что нормировка проведена успешно
features_norm_np.min(), features_norm_np.max()

(0.0, 1.0)

Разделяем данные для обучения и для тестирования

In [191]:
test_size = 0.3
stratify = targets_np
random_state = 0

features_train, features_test, targets_train, targets_test = train_test_split(
    features_norm_np,
    targets_np,
    test_size=test_size,
    stratify=stratify,
    random_state=random_state
)

Проверяем, что разделилось все правильно

In [192]:
# 42_000 * 0.7 = 29_400
features_train.shape, targets_train.shape

((29400, 784), (29400,))

In [193]:
# 42_000 - 29_400 = 12_600
features_test.shape, targets_test.shape

((12600, 784), (12600,))

Подгоним размерности для сверточных слоев

In [194]:
features_train = features_train.reshape(features_train.shape[0], 1, 28, 28)
features_test = features_test.reshape(features_test.shape[0], 1, 28, 28)

Переводим в тензоры

In [195]:
features_train_tn = torch.from_numpy(features_train).float()
targets_train_tn = torch.from_numpy(targets_train).long()

features_test_tn = torch.from_numpy(features_test).float()
targets_test_tn = torch.from_numpy(targets_test).long()

Проверяем, что все перевелось правильно

In [196]:
type(features_train_tn), type(targets_train_tn), type(features_test_tn), type(targets_test_tn)

(torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor)

Создаем Датасеты и Даталоудеры

In [197]:
batch_size = 64
num_workers = 2

train_ds = TensorDataset(features_train_tn, targets_train_tn)
test_ds = TensorDataset(features_test_tn, targets_test_tn)

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
    drop_last=True,
    pin_memory=True
)
test_loader = DataLoader(
    test_ds,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=True,
    pin_memory=True
)

Проверяем, что все создалось правильно

In [198]:
train_ds, test_ds

(<torch.utils.data.dataset.TensorDataset at 0x18fe13caba0>,
 <torch.utils.data.dataset.TensorDataset at 0x18fd2b864e0>)

In [199]:
train_loader, test_loader 

(<torch.utils.data.dataloader.DataLoader at 0x18fe13a6540>,
 <torch.utils.data.dataloader.DataLoader at 0x18fd2b87650>)

# Создаем Сам Классификатор

In [200]:
class DigitClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # Компоненты классификатора
        self.conv_block = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2) 
        )
        
        self.linear_block = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(128*7*7, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        x = self.conv_block(x)
        x = x.view(x.size(0), -1)
        x = self.linear_block(x)
        
        return x

# Обучение

Подготовка

In [201]:
model = DigitClassifier()

loss_function = nn.CrossEntropyLoss()

learning_rate = 0.001
weight_decay = 1e-4
optimizer = optim.Adam(
    params=model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay
)

exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

epochs = 25

Переведем все, что можно нужное устройство

In [202]:
if device.type == 'cuda':
    model.cuda()
    loss_function.cuda()

Главный цикл обучения

In [203]:
model.train()
for epoch in range(epochs):
    for images, labels in train_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        logits_ps = model(images)
        loss = loss_function(logits_ps, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    exp_lr_scheduler.step()

Валидация

In [204]:
correct = 0
test_size = len(test_loader.dataset)

with torch.no_grad():
    model.eval()
    for images, labels in test_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        logits_ps = model(images)
        loss = loss_function(logits_ps, labels)

        predictions = torch.argmax(logits_ps, dim=1)

        correct += (predictions == labels).sum().item()

Получившаяся точность

In [205]:
accuracy = correct / test_size
print(f'Test Accuracy: {accuracy * 100:.2f}')

Test Accuracy: 98.87


# Подготовка Данных Для Сдачи

Переводим в пандас

In [206]:
finaltest_np = finaltest.values
type(finaltest_np)

numpy.ndarray

Нормируем

In [207]:
finaltest_norm_np = finaltest_np / 255


Проверяем

In [208]:
finaltest_norm_np.min(), finaltest_norm_np.max()

(0.0, 1.0)

In [209]:
finaltest_norm_np.shape

(28000, 784)

Подгоняем размерности под сверточные слои

In [210]:
finaltest_norm_np = finaltest_norm_np.reshape(finaltest_norm_np.shape[0], 1, 28, 28)

Проверяем

In [211]:
finaltest_norm_np.shape

(28000, 1, 28, 28)

Переводим в тензор

In [212]:
finaltest_tn = torch.from_numpy(finaltest_norm_np).float()

Проверяем

In [213]:
type(finaltest_tn)

torch.Tensor

Производим прогнозирование на финальных данных

In [214]:
labels = []

model.eval()
with torch.no_grad():
    for data in finaltest_tn:
        data = data.to(device, non_blocking=True)
        
        logits_ps = model(data.unsqueeze(1))
        
        prediction = torch.argmax(logits_ps).item()
        
        labels.append(prediction)

Составление датайрейма для записи в файл

In [215]:
submission_df = pd.DataFrame({
    'ImageId': range(1, len(labels) + 1),
    'Label': labels
})

Проверка, что все занеслось правильно

In [216]:
submission_df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


Запись в файл для сдачи

In [217]:
submission_df.to_csv('submission.csv', index=False)